<a href="https://colab.research.google.com/github/BarbosaXBarbosa/BIG_S3/blob/main/PySpark_S3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Criando ambiente Spark:

## Instalando biblioteca
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("SenaiBS3").getOrCreate()

In [4]:
# Criando um DF manualmente através de listas
dados = [
    ("joao", 25, "analista", 3800),
    ("maria", 38, "analista", 5500),
    ("marcos", 22, "estagiario", 1000),
    ("fabio", 35, "assistente", 2500),
    ("luis", 40, "gerente", 11700),
    ("ana", 39, "gerente", 15900)
]

In [5]:
# Cabeçalho
colunas = ["nome", "idade", "cargo", "salario"]

In [6]:
# Criando DF Spark:
df = spark.createDataFrame(dados, colunas)

In [8]:
df.show()

+------+-----+----------+-------+
|  nome|idade|     cargo|salario|
+------+-----+----------+-------+
|  joao|   25|  analista|   3800|
| maria|   38|  analista|   5500|
|marcos|   22|estagiario|   1000|
| fabio|   35|assistente|   2500|
|  luis|   40|   gerente|  11700|
|   ana|   39|   gerente|  15900|
+------+-----+----------+-------+



In [9]:
df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- idade: long (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: long (nullable = true)



In [11]:
# Filtrar elementos do df:
df.where("cargo == 'gerente'").show()

+----+-----+-------+-------+
|nome|idade|  cargo|salario|
+----+-----+-------+-------+
|luis|   40|gerente|  11700|
| ana|   39|gerente|  15900|
+----+-----+-------+-------+



In [14]:
# Filtre apenas os funcionários que ganham menos de 3k
df.where("salario < 3000").show()

+------+-----+----------+-------+
|  nome|idade|     cargo|salario|
+------+-----+----------+-------+
|marcos|   22|estagiario|   1000|
| fabio|   35|assistente|   2500|
+------+-----+----------+-------+



In [15]:
dados = [
    ("luciana", 22, "assistente", 2700),
    ("carlos", 28, "analista", 5500)
]

In [16]:
df2 = spark.createDataFrame(dados, colunas)

In [17]:
df2.show()

+-------+-----+----------+-------+
|   nome|idade|     cargo|salario|
+-------+-----+----------+-------+
|luciana|   22|assistente|   2700|
| carlos|   28|  analista|   5500|
+-------+-----+----------+-------+



In [18]:
# Unindo os dois DataFrames
df = df.union(df2)

In [19]:
df.show()

+-------+-----+----------+-------+
|   nome|idade|     cargo|salario|
+-------+-----+----------+-------+
|   joao|   25|  analista|   3800|
|  maria|   38|  analista|   5500|
| marcos|   22|estagiario|   1000|
|  fabio|   35|assistente|   2500|
|   luis|   40|   gerente|  11700|
|    ana|   39|   gerente|  15900|
|luciana|   22|assistente|   2700|
| carlos|   28|  analista|   5500|
+-------+-----+----------+-------+



In [21]:
# Realizando algumas análises:

## Mostre a média salarial dos analistas dessa empresa:
df\
    .where("cargo == 'analista'")\
    .agg(                         ### Agreggate
        f.mean("salario").alias("media_salario"),
        f.sum("salario").alias("total_salario")
    )\
    .show()

+-----------------+-------------+
|    media_salario|total_salario|
+-----------------+-------------+
|4933.333333333333|        14800|
+-----------------+-------------+



In [22]:
# Importando arquivos para o Spark
url = '/content/drive/MyDrive/Colab Notebooks/Big_S3/empresas/part*'

empresas = spark.read.options(header=True).csv(
    url,
    sep=";",               ### Separator
    inferSchema=True
)

In [23]:
empresas.count()

4585679

In [28]:
empresas.show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|          0,00|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|          0,00|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|          0,00|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|                      50|          0,00|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|                      49|       4000,00|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|          0,00|    5|           NULL|
| 8992|SHIROMA VEIC

In [29]:
empresas.show(truncate=False)

+-----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj |razao_social                                                                                |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|306  |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0,00          |1    |NULL           |
|1355 |BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0,00          |5    |NULL           |
|4820 |REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E TABELIONATO E REGISTRO DE CONSTRATOS MARIT

In [30]:
empresas.show(10, False)

+----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj|razao_social                                                                                |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|306 |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0,00          |1    |NULL           |
|1355|BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0,00          |5    |NULL           |
|4820|REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E TABELIONATO E REGISTRO DE CONSTRATOS MARITIMOS|3

In [24]:
url = '/content/drive/MyDrive/Colab Notebooks/Big_S3/estabelecimentos/part*'

estab = spark.read.options(header=True).csv(
    url,
    sep=";",               ### Separator
    inferSchema=True
)

In [25]:
estab.count()

4836219

In [26]:
url = '/content/drive/MyDrive/Colab Notebooks/Big_S3/socios/part*'

socios = spark.read.options(header=True).csv(
    url,
    sep=";",               ### Separator
    inferSchema=True
)

In [27]:
socios.count()

2046430